In [741]:
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from sympy import Eq, solve_linear_system, Matrix, Function
from copy import deepcopy
import numpy as np
import sympy as sp

In [756]:
def get_math_operator(s:str, inverse=False):
    """ Need more operators! """
    if inverse:
        s = s.replace('+', '-').replace('*', '/')
    arr = []
    s = s.replace(" ", "")
    for i in range(len(s)):
        if s[i] in ['*', '+', '/', '-']:
            arr.append(s[i])
    return arr

def evaluate_op(x, y, op: str):
    if op == '*':
        x *= y
    elif op == '+':
        x += y
    elif op == '-':
        x -= y
    elif op == '/':
        x /= y
    return x

def sort_states(states):
    n = len(states)
    st = [0 for i in range(n)]
    ind = [int(str(i).replace(str(i)[0],"")) for i in states]
    for i, val in enumerate(states):
        st[ind[i]-1] = val
    return st

def diff_t(f, expressions):
    f.free_symbols  # Get variables
    df = ""
    for i in f.free_symbols:
        deriv = str(sp.diff(f, i)) + '*'
        if '1.0*' in deriv:
            deriv = deriv.replace('1.0*', '')
        elif '1*' in deriv:
            deriv = deriv.replace('1*', '')
        i_d = deriv + str(i) + 'd'
        df += i_d+"+"
    df = df[0:-1]  # Remove excess operator
    
    ## Get operators and variables from String
    Vd_operator = get_math_operator(df)
    Vd_variable = df.replace('+', '*').replace('-', '*').split('*')
    
    ## Get math operator priority
    hp = []  # High priority
    lp = []
    for i, val in enumerate(Vd_operator):
        if val in ['*', '/']:
            hp.append(i)
        elif val in ['+', '-']:
            lp.append(i)

    ## Change df from str to sym
    if lp == []:
        Vd_new = [expressions[str(df)]]
    else:
        # Get multiplication/division syms
        Vd_multi = []
        Vd_new = []
        i = 0
        for i in hp:
            var1 = expressions[Vd_variable[i]]
            var2 = expressions[Vd_variable[i+1]]
            Vd_multi.append(evaluate_op(var1, var2, Vd_operator[i]))
        # Add/substract syms
        for i, val in enumerate(lp):
            if Vd_multi == []:
                var1 = expressions[Vd_variable[i]]
                var2 = expressions[Vd_variable[i+1]]
            else:
                var1 = Vd_multi[i]
                var2 = Vd_multi[i+1]
            Vd_new.append(evaluate_op(var1, var2, Vd_operator[val]))
            
    df = sp.factor(Vd_new[0])
    return df

def get_sym_dict(Xd):
    n = len(Xd)
    g = Matrix([Xd[i].coeff(u) for i in range(n)])
    f = Xd - g*u
    states = sort_states(f.free_symbols)
    sym_dict = {f"x{i+1}d":Xd[i] for i in range(len(Xd))}
    sym_dict.update({f"x{i+1}":val for i, val in enumerate(states)})
    return sym_dict

def evaluate_eigenvalue(w):
    print(f"Eigenvalues are: {w}")
    complex = (np.iscomplex(ew)).all()
    if (np.real(w) < 0).all():
        if not complex:
            print("System is stable")
        else:
            print("System is stable focus")
    elif (np.real(w) > 0).all():
        print("System is unstable")
    elif (np.real(w) == 0).all():
        print("Eigenvalue is pure imaginary, stability analysis not possible for nonlinear systems!")
        
def get_relative_degree(A, y):
    n = len(A)
    g = Matrix([A[i].coeff(u) for i in range(n)])
    f = A - g*u
    states = sort_states(f.free_symbols)
    v = 1
    Lfh = y
    for i in range(n):
        LgLfh = g.T*Matrix([sp.diff(Lfh, i) for i in states])
        print(f"{v=} | LgLfh={LgLfh[0]} | Lfh={Lfh}")
        if not LgLfh[0]==0:
            break
        Lfh = f.T*Matrix([sp.diff(Lfh, i) for i in states])
        v += 1
    return v

def check_lyapunov_stability(V, Xd):
    # Derivate dV/dt
    # Substitute x in Vd
    # Check if negative definite
    # If semi negdef, do la salle
    return

def transform_and_check_stability(Xd, Z_dict):
    sym_dict = get_sym_dict(Xd)
    z1d = diff_t(Z_dict['z1'], get_sym_dict(Xd))
    z2d = diff_t(Z_dict['z2'], get_sym_dict(Xd))
    z3d = diff_t(Z_dict['z3'], get_sym_dict(Xd))

    for i in [x1, x2, x3]:
        z1d = z1d.subs(i, Z_dict[str(i)])
        z2d = z2d.subs(i, Z_dict[str(i)])
        z3d = z3d.subs(i, Z_dict[str(i)])

    # Find u
    eq1 = sp.Function('eq1')
    eq1 = Eq(z1d, v)
    Z_dict['u'] = sp.solve(eq1, u)[0]
    Z = Matrix([z2d, z3d])

    # Evaluate internal dynamics
    for i in range(len(Z)):
        Z[i] = Z[i].subs([(u, Z_dict['u']), (v, 0), (z1, 0)])

    display(Z)
    Y = sp.Matrix([i  for i in Z.free_symbols])
    A = Z.jacobian(Y)
    for i in Y:
        A = A.subs(i, 0)
    display(A)
    A = np.array(A).astype(np.float64)
    try:
        ew, ev = np.linalg.eig(A)
        evaluate_eigenvalue(ew)
    except TypeError:
        print("Eigenvalue on the imaginary axis, stability analysis not possible!")
    return

def get_z_sym(a, b, c):
    Z = [z1, z2, z3]
    Z_dict = {'z1': a, 'z2': b, 'z3': c}
    Z_copy = deepcopy(Z_dict)
    for i, val in enumerate(Z_copy.items()):
        if len(str(val[1]))>3:
            Vd_operator = get_math_operator(str(val[1]), inverse=True)
            Vd_variable = str(val[1]).replace(" ", '').replace('+', '*').replace('-', '*').split('*')
            temp = (i, val)
        else:
            Z_dict.update({str(val[1]): Z[i]})
    try:
        if temp:
            Z_dict.update({str(Vd_variable[0]): evaluate_op(Z[temp[0]], Z_dict[Vd_variable[1]], Vd_operator[0])})
    except NameError:
        pass
        
    return Z_dict

In [757]:
x1, x2, x3, u = sp.symbols('x1 x2 x3 u')
z1, z2, z3, v = sp.symbols('z1 z2 z3 v')

In [758]:
x1d = x2 - x3
x2d = -x1*x3 - 2*x2 + u
x3d = x1 + u
Xd = Matrix([x1d, x2d, x3d])

In [759]:
# Exercise 1a
y = x1
print(f"Relative degree is {get_relative_degree(Xd, y)}")

v=1 | LgLfh=0 | Lfh=x1
v=2 | LgLfh=0 | Lfh=Matrix([[x2 - x3]])
v=3 | LgLfh=-x1 - 2 | Lfh=Matrix([[-x1*x3 - x1 - 2*x2]])
Relative degree is 3


In [760]:
# Exercise 1b
y = x2
print(f"Relative degree is {get_relative_degree(Xd, y)}")
Z = get_z_sym(x2, x1, x3)
transform_and_check_stability(Xd, Z)

v=1 | LgLfh=1 | Lfh=x2
Relative degree is 1


⎡   -z₃    ⎤
⎢          ⎥
⎣z₂⋅z₃ + z₂⎦

⎡0  -1⎤
⎢     ⎥
⎣1  0 ⎦

Eigenvalues are: [0.+1.j 0.-1.j]
Eigenvalue is pure imaginary, stability analysis not possible for nonlinear systems!


In [761]:
# Exercise 1c
y = x3
print(f"Relative degree is {get_relative_degree(Xd, y)}")
Z = get_z_sym(y, x1, x2)
transform_and_check_stability(Xd, Z)

v=1 | LgLfh=1 | Lfh=x3
Relative degree is 1


⎡    z₃    ⎤
⎢          ⎥
⎣-z₂ - 2⋅z₃⎦

⎡0   1 ⎤
⎢      ⎥
⎣-1  -2⎦

Eigenvalues are: [-1. -1.]
System is stable focus


In [762]:
# Exercise 1d
y = x2 + x3
print(f"Relative degree is {get_relative_degree(Xd, y)}")
Z = get_z_sym(y, x1, x3)
transform_and_check_stability(Xd, Z)

v=1 | LgLfh=2 | Lfh=x2 + x3
Relative degree is 1


⎡     -2⋅z₃     ⎤
⎢               ⎥
⎢z₂⋅z₃   z₂     ⎥
⎢───── + ── - z₃⎥
⎣  2     2      ⎦

⎡ 0   -2⎤
⎢       ⎥
⎣1/2  -1⎦

Eigenvalues are: [-0.5+0.8660254j -0.5-0.8660254j]
System is stable focus


In [770]:
# Exercise 2
x1d = x3 - u
x2d = x1 + x2**2
x3d = x2 - 2*x3 + u
Xd = Matrix([x1d, x2d, x3d])

y = x2
print(f"Relative degree is {get_relative_degree(Xd, y)}")

v=1 | LgLfh=0 | Lfh=x2
v=2 | LgLfh=-1 | Lfh=Matrix([[x1 + x2**2]])
Relative degree is 2


In [771]:
# Lyapunov

# Exercise 1
x1d = -x1**3 + x1*x2**2
x2d = -x1**2*x2 - x2**3
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

# Exercise 2
x1d = 2*x2 - x1
x2d = -2*x1 - x2
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

# Exercise 3.3
x1d = -3*x1 - x1**3 - x1*x2
x2d = -2*x1**2 - x2 + x1**2
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

# Exercise 3.4
x1d = -40*x1*x2**2 + 10*x2 - 10*x1**3
x2d = 40*x1**3 - 10*x1
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

Vd=-x1**4 - x2**4
Vd=-x1**2 - x2**2
Vd=-x1**4 - 2*x1**2*x2 - 3*x1**2 - x2**2
Vd=-10*x1**2*(x1 - 2*x2)**2
